# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## <span style='color:#2656a3'> 🗒️ This notebook is divided into the following sections:

1. Load batch data.
2. Predict using model from Model Registry.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

First, we'll install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we'll proceed to import all the necessary libraries.

In [ ]:
# Importing the packages for the needed libraries for the Jupyter notebook
import joblib
import inspect 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

#%config InlineBackend.figure_format='retina'
#%matplotlib inline

## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

In [ ]:
# Importing the hopsworks module
import hopsworks

# Logging in to the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store() 

### <span style='color:#2656a3'> ⚙️ Feature View Retrieval

In [ ]:
# Retrieve the 'electricity_feature_view' feature view
feature_view = fs.get_feature_view(
    name='electricity_feature_view',
    version=1,
)

### <span style='color:#2656a3'> 🗄 Model Registry

In [ ]:
# Retrieve the model registry
mr = project.get_model_registry()

## <span style='color:#2656a3'> 📮 Retrieving model from Model Registry

In [ ]:
# Retrieving the TensorFlow model from the Model Registry
model = mr.get_model(
    name="electricity_price_prediction_model", 
    version=1,
)

# Downloading the model to a local directory
model_dir = model.download()

# Loading the saved TensorFlow model
loaded_model = tf.saved_model.load(model_dir)

# Accessing the serving function of the loaded model
serving_function = loaded_model.signatures["serving_default"]

## <span style='color:#2656a3'> ✨ Load Batch Data

In [ ]:
import datetime

# Calculating the start date as 5 days ago from the current date
start_date = datetime.datetime.now() - datetime.timedelta(days=5)

# Converting the start date to a timestamp in milliseconds
start_time = int(start_date.timestamp()) * 1000

In [ ]:
# Initializing batch scoring
feature_view.init_batch_scoring(1)

# Retrieving batch data from the feature view starting from the specified start time
df = feature_view.get_batch_data(
    start_time=start_time,
)

In [ ]:
# Sorting the DataFrame based on the 'timestamp' column
df.sort_values(["timestamp"], inplace=True)

# Dropping the 'day' and 'timestamp' columns from the DataFrame
df = df.drop(["day", "timestamp"], axis=1)

# Displaying the first 3 rows of the modified DataFrame
df.head(3)

## <span style='color:#2656a3'> 🤖 Making the predictions

In [ ]:
# Using the serving function to make predictions for the given DataFrame
x = serving_function(
    tf.constant(
        df.values.reshape(-1, df.shape[0], df.shape[1]), tf.float32)
)

# Extracting and converting the predictions to a NumPy array
prediction = x[list(x.keys())[0]].numpy() #.flatten().tolist()
prediction

In [ ]:
# Extracting and organizing the predicted prices for different regions from the batch predictions
price_dk1_pred = []

for batch_n in range(len(prediction)):
    batch = prediction[batch_n]
    for window_n in range(df.shape[0]):
        price_dk1_pred.append(batch[window_n][0])

In [ ]:
# Initializing serving
feature_view.init_serving(1)

# Accessing the transformation functions used in the serving configuration
td_transformation_functions = feature_view._batch_scoring_server._transformation_functions

In [ ]:
# Decoding the features from the predicted prices
preds = pd.DataFrame(data={'price_dk1': price_dk1_pred,})

# Extracting and decoding the transformation functions used in serving
res = {}
for feature_name in td_transformation_functions:
    if feature_name in ["price_dk1"]:
        td_transformation_function = td_transformation_functions[feature_name]
        sig, foobar_locals = inspect.signature(td_transformation_function.transformation_fn), locals()
        param_dict = dict([(param.name, param.default) for param in sig.parameters.values() if param.default != inspect._empty])
        if td_transformation_function.name == "min_max_scaler":
            preds[feature_name] = preds[feature_name].map(
                lambda x: x * (param_dict["max_value"] - param_dict["min_value"]) + param_dict["min_value"]
                )

# Applying a transformation to reverse the sign of the decoded features
preds = preds.apply(lambda x: -x)
preds

In [ ]:
# Plotting the predicted electricity prices for different regions
plt.plot(preds["price_dk1"],color='red', label='DK1 price prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(loc='upper left')

# Adjusting the size of the plot
fig = plt.gcf()
fig.set_size_inches(15, 5)

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Electricity price tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai